In [1]:
search_for = 136
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.09169435501098633
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 1048119951
type: [1, 1, 1, 1, 136] 136
cases of this type: 342102016
100000 31.12542533509251
200000 54.906516814715815
300000 56.48141210501456
400000 59.518095899404315
500000 49.27318995860609
600000 54.27059239022367
700000 61.49599147092876
800000 68.20123901128055
900000 69.03734980736624
1000000 75.4128280617076
1100000 83.47606195984643
1200000 91.65728462459762
1300000 32.34511441804316
1400000 91.27281916844187
1500000 96.36295672800958
1600000 97.12325590764104
1700000 96.33989523332045
1800000 100.3049180911545
1900000 107.12187934116436
2000000 104.96031274695711
2100000 106.57196324037652
2200000 104.40738127649306
2300000 106.66412611631911
2400000 101.03302386050726
2500000 87.29915474355043
2600000 31.508826957436785
2700000 100.78736755538918
2800000 99.99108325760398
2900000 99.98516843950357
3000000 96.525800

29400000 107.94132092876247
29500000 113.31363117441697
29600000 111.44521611932592
29700000 107.64692487527498
29800000 107.14504154053039
29900000 114.47334913479943
30000000 67.56887195891844
30100000 88.53370355237784
30200000 113.02860454402678
30300000 104.09032289665966
30400000 88.24647542173564
30500000 63.34323390120867
30600000 103.55868113499027
30700000 104.98825087586883
30800000 107.11858075344084
30900000 100.50055328149885
31000000 112.51115356583954
31100000 105.21284464696762
31200000 103.58965820713868
31300000 103.61203584249215
31400000 99.18627600615257
31500000 20.084589407447066
31600000 106.49116572042846
31700000 100.5268568956065
31800000 99.00955009365099
31900000 109.67420677628357
32000000 104.17876639421215
32100000 108.87027595621
32200000 104.0715754161574
32300000 103.63049556755004
32400000 109.01784363568753
32500000 63.423642116011564
32600000 87.76027940696336
32700000 112.61048823222416
32800000 106.85832539489003
32900000 112.2083416292964
33000

59300000 94.26747234283889
59400000 94.91629270584014
59500000 95.87430062822793
59600000 101.75636658299295
59700000 97.56806830211924
59800000 100.99110207055202
59900000 101.04046806738894
60000000 34.86973498679304
60100000 108.87323669265801
60200000 107.14662710424976
60300000 104.72653218998052
60400000 106.56327386500001
60500000 102.85520792846202
60600000 100.33835056293934
60700000 103.6947782246371
60800000 89.70510254246832
60900000 49.20247021556215
61000000 96.72554710948324
61100000 93.39777687403537
61200000 96.58993881378896
61300000 95.6470411259701
61400000 90.80873760836705
61500000 90.23200257089437
61600000 86.65226564092738
61700000 26.727504511696683
61800000 91.58902984807672
61900000 87.44782793379395
62000000 91.00043478455474
62100000 93.03059405221302
62200000 95.00285011325417
62300000 92.13366211315427
62400000 100.9962038171875
62500000 32.976380562293784
62600000 108.05334820541898
62700000 101.62279380770161
62800000 96.54927377902798
62900000 104.386

89400000 53.34525885664998
89500000 91.18758865826955
89600000 90.3416812140579
89700000 94.43350919927437
89800000 89.21257124507649
89900000 72.40564001637475
90000000 56.2656849476733
90100000 93.22690605473578
90200000 95.46353682309861
90300000 99.13104096630425
90400000 98.36022479267724
90500000 99.82805872067877
90600000 92.25112786732753
90700000 100.00078285436963
90800000 98.1633769186972
90900000 99.19582902031514
91000000 92.86535405870245
91100000 93.555126554544
91200000 89.93541869866651
91300000 44.32329406481022
91400000 86.37360576858431
91500000 94.11264589643379
91600000 90.62835704238537
91700000 86.49862014446556
91800000 73.27628857970221
91900000 39.72440084008662
92000000 85.14105270732885
92100000 86.68642179358972
92200000 86.54630158187149
92300000 87.56839753218449
92400000 67.74616755056226
92500000 57.95552956635099
92600000 93.14431680668615
92700000 91.20647372204041
92800000 93.22833978394736
92900000 97.76117775663376
93000000 95.91811621355211
93100

119000000 92.95124896338375
119100000 69.20207609417626
119200000 56.35877151269488
119300000 91.40927838178524
119400000 88.56778139844721
119500000 51.78341130551488
119600000 68.2557350674726
119700000 90.19408409579907
119800000 98.00842268194364
119900000 53.19835065421161
120000000 79.47332522032981
120100000 89.85338708005291
120200000 87.67655631818128
120300000 90.0690723985867
120400000 96.11980737167796
120500000 95.31370624041672
120600000 105.81284453775767
120700000 99.0541621501293
120800000 109.04232435967211
120900000 98.20738909877716
121000000 95.79667926568376
121100000 97.40447292108836
121200000 95.37800072153975
121300000 87.58795763564237
121400000 88.89347361535334
121500000 84.79778562737752
121600000 83.17623927398326
121700000 30.93300644551983
121800000 93.37879409816465
121900000 90.37804356332434
122000000 61.98666132297717
122100000 51.57134980332262
122200000 84.494187297185
122300000 89.89641542010018
122400000 48.094766537652596
122500000 75.935116894

...

953300000 1.512481295081367
953400000 0.7324569592106283
953500000 0.8173333867065727
953600000 1.676614849192369
953700000 1.6807557293699442
953800000 1.5627153532221258
953900000 1.4887347453504172
954000000 1.544820665796121
954100000 0.8565292631041739
954200000 1.66360673990311
954300000 1.9467130404174864
954400000 3.145315279855027
954500000 2.0707345175515894
954600000 1.372511065317138
954700000 0.5373558779559857
954800000 1.4606529807117754
954900000 1.5678075431390186
best so far: 0
type: [1, 2, 1, 17, 4] 136
cases of this type: 2515456
955000000 1.795396799117652
955100000 1.5800310236696442
955200000 1.4331245142317
955300000 1.2461539935540635
955400000 1.2393619072101354
955500000 1.313064138125054
955600000 1.4401949083319983
955700000 1.756879161739677
955800000 1.4411489334550995
955900000 1.1328912020710264
956000000 1.3370803885882423
956100000 1.1223141810873687
956200000 1.7449835628938826
956300000 1.4384824914974503
956400000 1.4918856470214168
956500000 1.217

979300000 0.3510004759520312
best so far: 0
type: [1, 2, 17, 2, 2] 136
cases of this type: 73984
979400000 0.3226328598634746
best so far: 0
type: [1, 2, 17, 4, 1] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 1, 2] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 2, 1] 136
cases of this type: 18496
best so far: 0
type: [1, 2, 68, 1, 1] 136
cases of this type: 9248
979500000 0.3018957666221082
best so far: 0
type: [1, 4, 1, 1, 34] 136
cases of this type: 5345344
979600000 1.6332096573431303
979700000 1.7896544277105795
979800000 1.6886521793361862
979900000 1.3475013301344931
980000000 1.7986718932773869
980100000 1.6186205986032902
980200000 1.442059463603874
980300000 1.3962659191668954
980400000 1.2802955834410965
980500000 1.389771856862586
980600000 1.2441736218200916
980700000 1.4000694449250957
980800000 1.5681095530325402
980900000 1.1776105960540646
981000000 1.5087968259031712
981100000 1.5848880517103872
981200000 1.0460618877815437
981300000 1.6

998800000 1.1004756467852588
998900000 1.27595123830617
999000000 0.8602053485348821
999100000 1.0729499749469513
999200000 1.2856282257172063
999300000 0.9962978964993954
999400000 0.9783259537259699
999500000 1.2637820637469357
999600000 1.0480061376275858
999700000 0.804730937309126
999800000 1.2225357270061645
999900000 1.2371553705922922
1000000000 1.068922573793416
1000100000 1.3149125561230959
1000200000 1.3032595825931668
1000300000 0.8642766015544912
1000400000 1.3167878407258828
1000500000 0.23702602475924686
1000600000 0.23591975204025042
1000700000 0.23678262110365697
1000800000 0.23618331921995284
1000900000 0.23483468077222108
1001000000 0.23583738481287295
1001100000 0.2416843504781842
1001200000 0.23309729678868982
1001300000 0.23312629531536033
1001400000 0.23518385571489336
1001500000 0.23221798474377264
1001600000 0.23133427137472365
1001700000 0.27768434517007673
1001800000 0.23575205738667315
1001900000 0.2330023298374825
1002000000 0.5770033335603059
1002100000 1.

1025600000 0.3632422985209902
1025700000 0.26955070377881524
1025800000 0.24489928793446158
1025900000 0.34303804231960705
1026000000 0.29177677219157816
1026100000 0.2069379748263796
1026200000 0.34003196691993404
1026300000 0.3043279600399852
1026400000 0.18151516289250982
1026500000 0.348432754912534
1026600000 0.31018717375640276
1026700000 0.1724077380362027
1026800000 0.3432424995045656
1026900000 0.3085737009030243
1027000000 0.22418631738379463
1027100000 0.3055200443060928
1027200000 0.30170350403666496
1027300000 0.21249168799435497
1027400000 0.2815887737360272
1027500000 0.3234223751036942
1027600000 0.23221998004034558
1027700000 0.31032195964902576
best so far: 0
type: [2, 1, 1, 17, 4] 136
cases of this type: 1257728
1027800000 0.323436505692933
1027900000 0.24800943124301303
1028000000 0.2278334021193438
1028100000 0.24586247159947755
1028200000 0.28353168647464516
1028300000 0.21861127252580723
1028400000 0.3152781799793144
1028500000 0.25488065018663075
1028600000 0.22

1045400000 0.042243735959114626
1045500000 0.04965560382189552
1045600000 0.04410044891304307
1045700000 0.03963645615575115
1045800000 0.04379611252485116
1045900000 0.034098252705826365
1046000000 0.0273181371828768
1046100000 0.010065319434607027
1046200000 0.02858921710756024
1046300000 0.028452273178929753
1046400000 0.032074007156924406
1046500000 0.027143950649573402
1046600000 0.025439448296660845
best so far: 0
type: [4, 1, 1, 2, 17] 136
cases of this type: 668168
1046700000 0.02688804581423402
1046800000 0.014097436820846132
1046900000 0.013183934835215409
1047000000 0.013630033835202456
1047100000 0.005090128501717249
1047200000 0.008282314095707071
1047300000 0.008919576850338777
best so far: 0
type: [4, 1, 1, 17, 2] 136
cases of this type: 78608
1047400000 0.005919434120190143
best so far: 0
type: [4, 1, 1, 34, 1] 136
cases of this type: 39304
best so far: 0
type: [4, 1, 2, 1, 17] 136
cases of this type: 334084
1047500000 0.0036375188320358597
1047600000 0.0052619260437965